# Lab CMBLenses

## Model

In [1]:
] activate "/home/jaimerz/julia_repos/CMBLensingInferenceTestProblem"

  Activating project at `~/julia_repos/CMBLensingInferenceTestProblem`


In [2]:
using CMBLensing, CMBLensingInferenceTestProblem
using CUDA, LaTeXStrings, MCMCDiagnosticTools, Plots, ProgressMeter, Random, Zygote
using LinearAlgebra, Statistics
ENV["LINES"] = 10;

In [3]:
using Revise
using MCHMC

In [4]:
prob = load_cmb_lensing_problem(storage = Array,
                                T = Float32,
                                Nside = 4);

MAP_joint: 100%|████████████████████████████████████████| Time: 0:00:49
  step:        20
  logpdf:      377.00
  α:           0.008522608
  CG:          2 iterations
  Linesearch:  9 bisections


In [5]:
target = CMBLensingTarget(prob)

CMBLensingTarget(50, [-0.30936679244041443 0.0 … 0.0 0.0; 0.0 -0.8185905814170837 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], MCHMC.var"#nlogp#26"{CMBLensingInferenceTestProblem.CMBLensingLogDensityProblem}(CMBLensingInferenceTestProblem.CMBLensingLogDensityProblem(BaseDataSet(d, Cf, Cn, Cn̂, M, M̂, B, B̂, logprior, Cϕ, Cf̃, D, G, L, Nϕ), [0.23362982273101807, 1.6145217418670654, -1.8872356414794922, -3.2767221927642822, 0.8644531965255737, 2.2458934783935547, -1.246073603630066, -2.639155864715576, 1.868626356124878, 3.251345157623291  …  -3.395840053599386e-9, 3.6597669339499816e-11, 4.677404241704153e-9, 1.211756028851596e-9, -5.810230518932258e-9, -2.3360031686792126e-9, 2.561251877253312e-9, -9.142808177209361e-10, -1.6094379124341003, 0.0], [0.0064222607761621475, 1.912354826927185, -2.338594675064087, -2.761254072189331, 0.3457961082458496, 2.8930203914642334, -1.2424980401992798, -3.303896427154541, 1.9643025398254395, 3.0344090461730957  …  -3.2332728494566254e-8, -3

## Sample

In [6]:
eps=0.00000001
L=sqrt(target.d)
spl=Sampler(;eps=eps, L=L)

Sampler(Settings(MersenneTwister(0), 1.0e-8, 7.0710678118654755, 0.0, 0.1931833275037836, 0.0005, 2000, 1000, 10, "LF"), MCHMC.Leapfrog)

In [7]:
init = MCHMC.Get_initial_conditions(spl, target; initial_x=target.Λmass*prob.Ωstart)

([-0.07227730751037598, -1.3216322660446167, 1.1293165683746338, 2.6822938919067383, 0.8535043597221375, -0.9474374055862427, 1.0418479442596436, -1.855589509010315, 1.6437256336212158, 0.340961754322052  …  -7.0131552e7, -446595.09375, 4.384656e7, -1.4786851e7, 1.26009488e8, -2.944091e7, -2.384044e7, -1.1522784e7, -1.6094379124341003, 0.0], [-0.020216939970850945, 0.01078077033162117, 0.1290055513381958, -0.011107100173830986, -0.044468484818935394, 0.014892672188580036, -0.1695379763841629, -0.22515244781970978, 0.08867604285478592, -0.16112224757671356  …  -0.07660537213087082, -0.21502257883548737, -0.10700489580631256, 0.168582022190094, -0.05181819945573807, -0.027390334755182266, -0.3986780047416687, 0.02819964848458767, -0.02784493005762052, -0.10787566314429173], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -Inf, -Inf], -Inf, 0.0)

In [8]:
init = MCHMC.Get_initial_conditions(spl, target; initial_x=prob.Ωstart)

([0.23362982273101807, 1.6145217418670654, -1.8872356414794922, -3.2767221927642822, 0.8644531965255737, 2.2458934783935547, -1.246073603630066, -2.639155864715576, 1.868626356124878, 3.251345157623291  …  -3.395840053599386e-9, 3.6597669339499816e-11, 4.677404241704153e-9, 1.211756028851596e-9, -5.810230518932258e-9, -2.3360031686792126e-9, 2.561251877253312e-9, -9.142808177209361e-10, -1.6094379124341003, 0.0], [-0.19302482903003693, 0.03811245039105415, -0.30685955286026, -0.03418673947453499, 0.03091432712972164, 0.09038689732551575, -0.10884086787700653, 0.3196006119251251, 0.05967727676033974, 0.09748073667287827  …  -0.100493423640728, -0.04102863371372223, -0.07725255191326141, 0.18768788874149323, -0.15252205729484558, -0.05798185244202614, -0.1404091864824295, -0.07826074212789536, -0.04404509640588234, 0.07838161447649154], [-3.002088817538606e-7, 3.2172076203096367e-7, -5.61635943086003e-7, -5.00593102970015e-7, 1.5009730702786328e-7, 1.0105396768267383e-6, -7.8160603322885

In [9]:
MCHMC.step(spl, target, init)

LoadError: MethodError: no method matching step(::Sampler, ::CMBLensingTarget, ::Tuple{Field-(f°,ϕ°,θ)-Tuple{LambertQUMap{Array{Float32, 3}}, LambertMap{Array{Float32, 2}}, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(r = 1, Aϕ = 2)}}}}, Field-(f°,ϕ°,θ)-Tuple{LambertQUMap{Array{Float32, 3}}, LambertMap{Array{Float32, 2}}, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(r = 1, Aϕ = 2)}}}}, Field-(f°,ϕ°,θ)-Tuple{LambertQUMap{Array{Float32, 3}}, LambertMap{Array{Float32, 2}}, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(r = 1, Aϕ = 2)}}}}, Float64, Float64})

In [10]:
samples = Sample(spl, target; num_steps=100)

LoadError: DomainError with -1.1547295280235:
log1p will only return a complex result if called with a complex argument. Try log1p(Complex(x)).

In [11]:
inv_Λmass = Diagonal(Vector(diag(pinv(target.Λmass))))

50×50 Diagonal{Float64, Vector{Float64}}:
 -3.23241    ⋅         ⋅        ⋅    ⋅   …   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
   ⋅       -1.22161    ⋅        ⋅    ⋅       ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
   ⋅         ⋅       -1.67113   ⋅    ⋅       ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋮                                      ⋱            ⋮                   
   ⋅         ⋅         ⋅        ⋅    ⋅       ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅ 
   ⋅         ⋅         ⋅        ⋅    ⋅       ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0

In [12]:
t_samples = [inv_Λmass * sample for sample in samples]

LoadError: UndefVarError: `samples` not defined

## Plotting

In [13]:
Plots.default(fmt=:png, dpi=120, size=(500,300), legendfontsize=10)

In [14]:
rs = [sample[end] for sample in samples]
Aϕs = [sample[end-1] for sample in samples];

LoadError: UndefVarError: `samples` not defined

In [15]:
println(mean(exp.(rs)), " ", std(exp.(rs)))

LoadError: UndefVarError: `rs` not defined

In [16]:
println(mean(exp.(Aϕs)), " ", std(exp.(Aϕs)))

LoadError: UndefVarError: `Aϕs` not defined

In [17]:
plot(exp.(rs), label=L"r", xlabel="step")
plot!(exp.(Aϕs), label=L"A_\phi")

LoadError: UndefVarError: `rs` not defined

In [18]:
plot(histogram(exp.(rs), xlabel=L"r", label=nothing, lw=1),
    histogram(exp.(Aϕs), xlabel=L"A_\phi", label=nothing, lw=1))

LoadError: UndefVarError: `rs` not defined